In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose solution and drawing utilities.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Open the default webcam (0 for the default camera).
cap = cv2.VideoCapture(0)

print("Starting webcam feed. Press 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Convert the frame from BGR (OpenCV format) to RGB (MediaPipe format).
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to extract pose landmarks.
    results = pose.process(image_rgb)

    # If landmarks are detected, draw them on the frame.
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
        )

    # Display the frame.
    cv2.imshow("Pose Detection", frame)

    # Press 'q' to exit the loop.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window.
cap.release()
cv2.destroyAllWindows()


Starting webcam feed. Press 'q' to quit.


KeyboardInterrupt: 

In [2]:
import math

def calculate_angle(a, b, c):
    """
    Calculates the angle at point b formed by points a and c.
    a, b, c are (x, y) tuples for their coordinates.
    Returns the angle in degrees.
    """
    ba = (a[0] - b[0], a[1] - b[1])  # Vector from b to a
    bc = (c[0] - b[0], c[1] - b[1])  # Vector from b to c

    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)

    # Avoid division by zero
    if mag_ba == 0 or mag_bc == 0:
        return 0.0

    # Angle in degrees
    angle = math.degrees(math.acos(dot_product / (mag_ba * mag_bc)))
    return angle


In [ ]:
import cv2
import mediapipe as mp
import math

def calculate_angle(a, b, c):
    """
    Calculates the angle at point b (in degrees) formed by points a, b, and c.
    a, b, c: tuples (x, y)
    """
    ba = (a[0] - b[0], a[1] - b[1])
    bc = (c[0] - b[0], c[1] - b[1])
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    if mag_ba == 0 or mag_bc == 0:
        return 0.0
    angle = math.degrees(math.acos(dot_product / (mag_ba * mag_bc)))
    return angle

def calculate_torso_angle(shoulder, hip):
    """
    Calculates the angle (in degrees) between the vector from hip to shoulder and the vertical axis.
    A perfectly vertical torso will yield an angle close to 0.
    shoulder, hip: tuples (x, y)
    """
    dx = shoulder[0] - hip[0]
    dy = hip[1] - shoulder[1]  # invert y because pixel y increases downward
    angle = math.degrees(math.atan2(dx, dy))
    return abs(angle)  # Absolute value; 0 means vertical

# Initialize MediaPipe Pose and drawing utilities.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

# Parameters for squat detection
squat_threshold = 90      # Knee angle below which the person is in squat position.
standing_threshold = 160  # Knee angle above which the person is considered standing.
upright_threshold = 15    # Torso angle (in degrees) close to 0 means upright.

# Variables for rep counting
in_squat = False
rep_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Convert frame for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        landmarks = results.pose_landmarks.landmark
        h, w, _ = frame.shape

        # Get left side landmarks (you can add right side similarly)
        left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h)
        left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * w,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * h)
        left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * w,
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * h)
        left_ankle = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * w,
                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * h)

        # Calculate knee angle at left knee
        knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        # Calculate torso angle (angle between vector from hip to shoulder and vertical)
        torso_angle = calculate_torso_angle(left_shoulder, left_hip)

        # Display the angles on the frame
        cv2.putText(frame, f"Knee Angle: {int(knee_angle)}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(frame, f"Torso Angle: {int(torso_angle)}", (50, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(frame, f"Reps: {rep_count}", (50, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)

        # Check for squat phase
        if knee_angle < squat_threshold:
            in_squat = True
            cv2.putText(frame, "Correct Squat position", (50, 170),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        # Check for standing upright after squat
        elif in_squat and knee_angle > standing_threshold:
            if torso_angle < upright_threshold:
                rep_count += 1
                cv2.putText(frame, "Upright! Rep complete", (50, 210),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            else:
                cv2.putText(frame, "Not upright! Stand straight", (50, 210),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            in_squat = False

    cv2.imshow("Squat and Upright Check", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [3]:
import cv2
import mediapipe as mp
import math

def calculate_angle(a, b, c):
    """
    Calculate the angle at point b formed by points a, b, and c.
    Points a, b, c should be given as (x, y) tuples.
    Returns the angle in degrees.
    """
    ba = (a[0] - b[0], a[1] - b[1])
    bc = (c[0] - b[0], c[1] - b[1])
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    mag_ba = math.sqrt(ba[0] ** 2 + ba[1] ** 2)
    mag_bc = math.sqrt(bc[0] ** 2 + bc[1] ** 2)
    
    # Protect against division by zero.
    if mag_ba == 0 or mag_bc == 0:
        return 0.0

    angle = math.degrees(math.acos(dot_product / (mag_ba * mag_bc)))
    return angle

# Initialize MediaPipe Pose and Drawing utilities.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Open the default webcam.
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Convert the image from BGR to RGB.
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
        )

        landmarks = results.pose_landmarks.landmark
        h, w, _ = frame.shape

        # Extract coordinates for left side: shoulder, hip, knee, ankle.
        left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * w,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * h)
        left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x * w,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * h)
        left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x * w,
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y * h)
        left_ankle = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x * w,
                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y * h)

        # Extract coordinates for right side: shoulder, hip, knee, ankle.
        right_shoulder = (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * w,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * h)
        right_hip = (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x * w,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * h)
        right_knee = (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x * w,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y * h)
        right_ankle = (landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x * w,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y * h)

        # Calculate hip angles (shoulder, hip, ankle) for both sides.
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_ankle)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_ankle)

        # Calculate knee angles (hip, knee, ankle) for both sides.
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

        # Average the hip angles and knee angles for a more balanced evaluation.
        avg_hip_angle = (left_hip_angle + right_hip_angle) / 2.0
        avg_knee_angle = (left_knee_angle + right_knee_angle) / 2.0

        # Display calculated angles.
        cv2.putText(frame, f"Hip Angle: {int(avg_hip_angle)}", 
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Knee Angle: {int(avg_knee_angle)}", 
                    (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Provide feedback:
        if 170 <= avg_hip_angle <= 185 and 170 <= avg_knee_angle <= 185:
            feedback = "Good plank!"
            color = (0, 255, 0)  # Green for good form.
        else:
            feedback = "Keep hips and knees straight!"
            color = (0, 0, 255)  # Red for corrections.

        cv2.putText(frame, feedback, 
                    (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Display the webcam feed.
    cv2.imshow("Plank Detection", frame)

    # Press 'q' to exit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [5]:
print('hi im alyster')

hi im alyster


In [6]:
print('hi im alyster')

hi im alyster
